In [1]:
from sklearn.svm import LinearSVC
from sklearn import preprocessing
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import collections
import os
import seaborn
seaborn.set()
from sklearn.externals import joblib

In [2]:
svc = joblib.load('svc.pkl')
encoded = np.load('./npy_data/data_encoded_d.npy')
varvals = np.load('./npy_data/varvals_d.npy')
pathdataOH = np.load("./npy_data/path_data_oh_d.npy")
oldpath = np.load('./npy_data/old_path_d.npy')

In [3]:
varvals

array([ 2.,  2., 11., ..., 31., 37., 42.])

In [4]:
varvals[20431281]

3.0

In [8]:
oldpath[13933181]

11199240

In [12]:
# retrieve all the nonzero coefficients and zip them with their respective indices
nonzeroes = np.nonzero(svc.coef_[0])[0]
coefs = zip(nonzeroes, svc.coef_[0][nonzeroes])

# sort the coefficients by their value, instead of index
coefs.sort(key = lambda x: x[1], reverse=True)

for coef in coefs:
    print coef
    print "Old path: ", oldpath[coef[0]]

(20674455, 0.5919716238171446)
Old path:  16565714
(5467982, 0.08607034243323428)
Old path:  4367107
(13933181, 0.0520436158134867)
Old path:  11199240
(13235098, 0.04390505706562038)
Old path:  10628375
(24884118, 0.038514661882012365)
Old path:  19959359
(12110356, 0.02393485511986009)
Old path:  9762973
(10654598, 0.02001787436042732)
Old path:  8592672
(18635721, 0.0040016483120448665)
Old path:  14907399
(893330, -0.01879369153441716)
Old path:  735742
(25117658, -0.022580767904878857)
Old path:  20145028
(6214939, -0.03070315360121059)
Old path:  4973456
(8580740, -0.0634672561462652)
Old path:  6951235
(12752822, -0.11724411303521072)
Old path:  10266270
(2147838, -0.11800596215033615)
Old path:  1736468
(15438902, -0.1252833349384073)
Old path:  12361751
(14379751, -0.13125359246731855)
Old path:  11533338
(14043795, -0.16198974801925214)
Old path:  11275501


In [25]:
maxCoef = np.absolute(svc.coef_).max()

idxM = np.argmax(np.absolute(svc.coef_))
numnz = np.nonzero(svc.coef_)[1].shape
idxNZus = np.nonzero(svc.coef_)[1]
coefs = svc.coef_[0,:]

nnzcoefs = coefs[idxNZus]

idxSort = np.argsort(np.absolute(nnzcoefs))
idxSort = np.flipud(idxSort)
idxNZ = idxNZus[idxSort]

coefPaths = pathdataOH[idxNZ]

tile_path = np.trunc(coefPaths/(16**5))
tile_step = np.trunc((coefPaths - tile_path*16**5)/2)
tile_phase = np.trunc((coefPaths- tile_path*16**5 - 2*tile_step))

In [29]:
print(tile_path, tile_step, tile_phase)

(array([0., 0., 0.]), array([30., 30., 30.]), array([0., 0., 0.]))


In [10]:
vhex = np.vectorize(hex)

In [11]:
vectorizedPath = vhex(tile_path.astype('int'))
vectorizedStep = vhex(tile_step.astype('int'))
vectorizedPhase = vhex(tile_phase.astype('int'))

In [30]:
tile_loc = np.column_stack((tile_path, tile_step))
print(tile_loc)
print(nnzcoefs[idxSort])
print(oldpath[idxNZ])
print(varvals[idxNZ])

[[ 0. 30.]
 [ 0. 30.]
 [ 0. 30.]]
[ 0.6158557  -0.49575757 -0.0089541 ]
[12016733 12016733  3648813]
[3. 2. 4.]


In [13]:
def tileSearch(arg):
    vecpath = str(vectorizedPath[int(arg)])
    vecpath = vecpath[2:].zfill(4)
    try:
        proc = subprocess.check_output("cat " + args.assembly_fwi + " | grep :" + vecpath, shell=True)
        return proc
    except CalledProcessError as e:
        return "Assembly index file not found or `cat` command not available. Continuing..."
    

In [14]:
tileSearch(12559433)

IndexError: index 12559433 is out of bounds for axis 0 with size 3